In [208]:
from skimage import io
# mat = io.imread("/home/zhizizhang/Documents/Mask_RCNN/gisdata/JingDeZhen_11/JingDeZhen_11_result_dom.tif")
mat = io.imread("/home/zhizizhang/Documents/Mask_RCNN/gisdata/20210723_JingDeZhen_02_waibao_shp/JingDeZhen_02_result_dom.tif")


In [209]:
import geopandas as gpd
from matplotlib import pyplot as plt

# sh_file = "/home/zhizizhang/Documents/Mask_RCNN/gisdata/JingDeZhen_11/JingDeZhen_11.shp"
sh_file = "/home/zhizizhang/Documents/Mask_RCNN/gisdata/JingDeZhen_02/JingDeZhen02_shp.shp"

data = gpd.read_file(sh_file)#读取磁盘上的矢量文件 shp


In [210]:
dataBounds = data.total_bounds
imgShape = mat.shape
miniShape = (int(imgShape[0]/10),int(imgShape[1]/10),imgShape[2])

In [211]:
def trans2xy(pos,imgShape,dataBounds):
	lon=pos[0]
	lat=pos[1]
	height = imgShape[0]-1 #图像高
	width = imgShape[1]-1 #图像宽
	(minLon,minLat,maxLon,maxLat) = dataBounds
	scaleX = ((maxLon-minLon) * 360000) / width #X轴上每像素代表的经度秒数；
	scaleY =  ((maxLat-minLat) * 360000) / height #Y轴上每像素代表的经度秒数；
	screenX = (lon-minLon) * 360000 / scaleX
	screenY = (maxLat-lat) * 360000 / scaleY
	return (int(screenX),int(screenY))

In [212]:
import json
# data.to_file('test.json', driver='GeoJSON', encoding="utf-8")
sdata=data.to_json()
jsondata=json.loads(sdata)

# print(type(jsondata))
# print(jdata['features'][490])
# jsondata['features'][491]

In [213]:
jsondata['features'][491]['geometry']['coordinates'][0]   # 第一个0 代表第一段 ，第二个0 代表第一个点
imgShape

(21004, 14004, 3)

## 经纬度转像素

In [214]:
pixeldata=jsondata
for idf,region in enumerate(jsondata['features']):
    if region['geometry']!=None:
        for idp,point in enumerate(region['geometry']['coordinates'][0]):
            pixeldata['features'][idf]['geometry']['coordinates'][0][idp]=trans2xy(point,imgShape,dataBounds)
            

## 图像切割并调整point

In [215]:
import os
output_DIR = './../../datasets/segBuildings/'
os.makedirs(output_DIR, exist_ok=True)

for idf,region in enumerate(pixeldata['features']):
    if region['geometry']!=None:
        #计算每个region区域并切割图像
        regionPoints = region['geometry']['coordinates'][0]
        (minX,maxY) = (min(regionPoints,key=lambda tup:tup[0])[0],max(regionPoints,key=lambda tup:tup[1])[1])
        (maxX,minY) = (max(regionPoints,key=lambda tup:tup[0])[0],min(regionPoints,key=lambda tup:tup[1])[1])

        # 扩展边界
        padding = 250
        minX = minX-padding if minX>=padding else minX
        minY = minY-padding if minY>=padding else minY
        maxX = maxX+padding if maxX<imgShape[1] -padding else maxX
        maxY = maxY+padding if maxY<imgShape[0] -padding else maxY
         
        pic_slice=mat[minY:maxY,minX:maxX,:]
        
        # 保持切片
        io.imsave(output_DIR+str(idf)+'.jpg',pic_slice)
        
        #更新json信息
        #添加 img = [image_name]
        pixeldata['features'][idf]['img']=str(idf)+'.jpg'
        
        # 当前区域所有左边转换为切片零点
        if region['geometry']!=None:
            for idp,point in enumerate(region['geometry']['coordinates'][0]):
                cp=pixeldata['features'][idf]['geometry']['coordinates'][0][idp]
                pixeldata['features'][idf]['geometry']['coordinates'][0][idp]=(cp[0]-minX + padding,cp[1]-minY + padding)
                

## 保存JSON

In [216]:
with open(output_DIR+"annotation.json",'w',encoding='utf-8') as json_file:
        json.dump(pixeldata,json_file,ensure_ascii=False)

## 调试备用代码，shp json文件读取与保存

In [217]:
## 字典 保存为json
# with open("./test.json",'w',encoding='utf-8') as json_file:
#         json.dump(pixeldata,json_file,ensure_ascii=False)

## shp保存为json
# data.to_file('test.json', driver='GeoJSON', encoding="utf-8")


In [218]:
## json 读取为 geopands
# shp = gpd.read_file('test.json')
# dd=shp.drop(['id'],axis=1)
# dd